In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''%データセットのダウンロード
!pip install gdown

import gdown
file_id = "1-EyM2kIj24P6DtT-swZP8DLyBAPU1PkU"
url = f"https://drive.google.com/uc?id={file_id}"
output = "dataset.zip"
gdown.download(url, output, quiet=False)
!unzip "dataset.zip"
'''

'%データセットのダウンロード\n!pip install gdown\n\nimport gdown\nfile_id = "1-EyM2kIj24P6DtT-swZP8DLyBAPU1PkU"\nurl = f"https://drive.google.com/uc?id={file_id}"\noutput = "dataset.zip"\ngdown.download(url, output, quiet=False)\n!unzip "dataset.zip"\n'

In [ ]:
#import libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
print(torch.__version__)
torch.manual_seed(0)


1.8.1+cu101


In [574]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
LReLU_alpha = 0.2
mapping_lamda = 0.01

#https://github.com/yuuho/stylegans-pytorch/blob/master/network/stylegan2.py　を参考に
from torch.nn import functional as F
class modulated_conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
        super(modulated_conv2d, self).__init__()
        self.padding, self.stride = padding, stride

        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size))
        torch.nn.init.normal_(self.weight.data, mean=0.0, std=1.0)
        #self.weight = nn.Parameter(torch.zeros(out_channels, in_channels, kernel_size, kernel_size))
        self.bias = nn.Parameter(torch.zeros(1,out_channels,1,1))
        self.weight_scaler = 1 / (in_channels * kernel_size*kernel_size)**0.5

    def forward(self,x,style,shape = None,web = True,style_2 = None):
        oC, iC, kH, kW = self.weight.shape
        if shape is None:
            N, iC, H, W = x.shape
        else:
            N, iC, H, W = shape

        if web:
            modulated_weight = self.weight.view(1,oC,iC,kH,kW) * self.weight_scaler 
            #modulated_weight = modulated_weight.repeat(N,1,1,1,1)
            x = x * style.view(N,iC,1,1)
            x = F.conv2d(x.view(1,N*iC,H,W), modulated_weight.view(N*oC,iC,kH,kW),
                    padding=self.padding, stride=self.stride, groups=N).view(N,oC,H,W)
            modulated_weight = modulated_weight * style_2.view(N,1,iC,1,1)
            demod_norm = 1 / torch.sqrt((modulated_weight * modulated_weight).sum([2,3,4])  + 1e-8)
            out = x * demod_norm.view(N, oC, 1, 1) + self.bias   

        else:
            modulated_weight = self.weight_scaler*self.weight.view(1,oC,iC,kH,kW) \
                                    * style.view(N,1,iC,1,1) 

            demod_norm = 1 / torch.sqrt((modulated_weight * modulated_weight).sum([2,3,4]) + 1e-8) # (N, oC)
            demodulated_weight = modulated_weight * demod_norm.view(N, oC, 1, 1, 1) 
            #demodulated_weight = modulated_weight
            out = F.conv2d(x.view(1,N*iC,H,W), demodulated_weight.view(N*oC,iC,kH,kW),
                    padding=self.padding, stride=self.stride, groups=N).view(N,oC,H,W) + self.bias
            
        return out

def alternative_Upsample(image,input_size):
    batches, channels, h, w = input_size

    x = image.view(batches, channels, h * w, 1)
    x = torch.cat((x,x),3)
    x = x.view(batches, channels, h, w * 2)
    x = torch.cat((x,x),3)
    x = x.view(batches, channels, h * 2, w * 2)
    return x
# Define model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        dimensions = [256,256,256,256,128,128,128,64,64,64,32,32]
        self.learning_const = torch.randn((1,dimensions[1],4,4),requires_grad=True, device = device)
        self.mapping_network = self.generate_mapping_network(dimensions[0])

        self.affine_4_0 = self.make_latent_to_style(dimensions[0],dimensions[1])
        self.block_4_0 = self.generate_block(4,dimensions[1],dimensions[2])
        self.noise_4_0 = torch.zeros((1,),requires_grad=True, device = device)
        self.affine_4_1 = self.make_latent_to_style(dimensions[0],dimensions[2])
        self.block_4_1 = self.generate_block(4,dimensions[2],dimensions[3])
        self.noise_4_1 = torch.zeros((1,),requires_grad=True, device = device)
        self.to_rgb_4 = nn.Conv2d(dimensions[3],3,kernel_size=1,stride=1,padding=0)

        self.affine_8_0 = self.make_latent_to_style(dimensions[0],dimensions[3])
        self.block_8_0 = self.generate_block(8,dimensions[3],dimensions[4],True)
        self.noise_8_0 = torch.zeros((1,),requires_grad=True, device = device)
        self.affine_8_1 = self.make_latent_to_style(dimensions[0],dimensions[4])
        self.block_8_1 = self.generate_block(8,dimensions[4],dimensions[5])
        self.noise_8_1 = torch.zeros((1,),requires_grad=True, device = device)
        self.to_rgb_8 = nn.Conv2d(dimensions[5],3,kernel_size=1,stride=1,padding=0)

        self.affine_16_0 = self.make_latent_to_style(dimensions[0],dimensions[5])
        self.block_16_0 = self.generate_block(16,dimensions[5],dimensions[6],True)
        self.noise_16_0 = torch.zeros((1,),requires_grad=True, device = device)
        self.affine_16_1 = self.make_latent_to_style(dimensions[0],dimensions[6])
        self.block_16_1 = self.generate_block(16,dimensions[6],dimensions[7])
        self.noise_16_1 = torch.zeros((1,),requires_grad=True, device = device)
        self.to_rgb_16 = nn.Conv2d(dimensions[7],3,kernel_size=1,stride=1,padding=0)

        self.affine_32_0 = self.make_latent_to_style(dimensions[0],dimensions[7])
        self.block_32_0 = self.generate_block(32,dimensions[7],dimensions[8],True)
        self.noise_32_0 = torch.zeros((1,),requires_grad=True, device = device)
        self.affine_32_1 = self.make_latent_to_style(dimensions[0],dimensions[8])
        self.block_32_1 = self.generate_block(32,dimensions[8],dimensions[9])
        self.noise_32_1 = torch.zeros((1,),requires_grad=True, device = device)
        self.to_rgb_32 = nn.Conv2d(dimensions[9],3,kernel_size=1,stride=1,padding=0)

        self.affine_64_0 = self.make_latent_to_style(dimensions[0],dimensions[9])
        self.block_64_0 = self.generate_block(64,dimensions[9],dimensions[10],True)
        self.noise_64_0 = torch.zeros((1,),requires_grad=True, device = device)
        self.affine_64_1 = self.make_latent_to_style(dimensions[0],dimensions[10])
        self.block_64_1 = self.generate_block(64,dimensions[10],dimensions[11])
        self.noise_64_1 = torch.zeros((1,),requires_grad=True, device = device)
        self.to_rgb_64 = nn.Conv2d(dimensions[11],3,kernel_size=1,stride=1,padding=0)
        for i in range(5):
            nn.init.normal_(getattr(self, 'to_rgb_{0}'.format(2 ** (i + 2))).weight, 0.0, 1.0 / getattr(self, 'to_rgb_{0}'.format(2 ** (i + 2))).weight.shape[1] ** 0.5)
            nn.init.constant_(getattr(self, 'to_rgb_{0}'.format(2 ** (i + 2))).bias, 0.0)
            
            for j in range(2):
                nn.init.normal_(getattr(self, 'affine_{0}_{1}'.format(2 ** (i + 2),j)).weight, 0.0, 1.0)
                nn.init.constant_(getattr(self, 'affine_{0}_{1}'.format(2 ** (i + 2),j)).bias, 1.0)         

    def generate_mapping_network(self,dimension = 512,number_of_layer = 1):
       mapping_network = nn.Sequential()
       for i in range(number_of_layer):
           mapping_network.add_module('mapping_fc{0}'.format(i), nn.Linear(dimension,dimension))
           mapping_network.add_module('mapping_lrelu{0}'.format(i), nn.LeakyReLU())
           nn.init.normal_(getattr(mapping_network, 'mapping_fc{0}'.format(i)).weight, 0.0, 1.0)
           nn.init.constant_(getattr(mapping_network, 'mapping_fc{0}'.format(i)).bias, 0)
       return mapping_network

    def generate_block(self,image_size = 4,in_dimension = 512, out_dimension = 512,upscaling = False):
        return modulated_conv2d(in_dimension,out_dimension,kernel_size=3,stride=1,padding=1)
        #return torch.nn.Conv2d(in_dimension,out_dimension,kernel_size=3,stride=1,padding=1)
    def make_latent_to_style(self,latent_dimension = 512, target_dimension = 512):

        return nn.Linear(latent_dimension, target_dimension)
    def forward(self, z, stage = 1 ,alpha = 0, batches = 1,web = False):
        style = self.mapping_network(z) * mapping_lamda
        if web:
            #mapping_network_2 = self.mapping_network_2(z)
            style_2 = self.mapping_network(z) * mapping_lamda

        x = self.learning_const.repeat(int(batches),1,1,1)
        for i in range(stage):
            if i != 0:
                if web:
                    x = alternative_Upsample(x,(1,-1,2 ** (i + 1),2 ** (i + 1)))
                else:
                    x = torch.nn.functional.interpolate(x,scale_factor=2, mode='nearest')
            for j in range(2):
                if not (i == 0 and j == 0):
                    affined_style = getattr(self, 'affine_{0}_{1}'.format(2 ** (i + 2),j))(style)
                    if web:
                        affined_style_2 = getattr(self, 'affine_{0}_{1}'.format(2 ** (i + 2),j))(style_2)
                    if web:
                        x = getattr(self, 'block_{0}_{1}'.format(2 ** (i + 2),j))(x,affined_style,shape = (1,-1,2 ** (i + 2),2 ** (i + 2)),style_2 = affined_style_2,web = True)
                    else:
                        x = getattr(self, 'block_{0}_{1}'.format(2 ** (i + 2),j))(x,affined_style,shape = (1,-1,2 ** (i + 2),2 ** (i + 2)),web = False)
                    x = F.leaky_relu(x,LReLU_alpha)
                    #x = x + torch.randn(x.shape, device = device) *  getattr(self, 'noise_{0}_{1}'.format(2 ** (i + 2),j))

        x_out = getattr(self, 'to_rgb_{0}'.format(2 ** (stage + 1)))(x)

        if alpha != 0:
            if web:
                x = alternative_Upsample(x,(1,-1,2 ** (stage + 1),2 ** (stage + 1)))
            else:
                x = torch.nn.functional.interpolate(x,scale_factor=2, mode='nearest')
            for j in range(2):
                affined_style = getattr(self, 'affine_{0}_{1}'.format(2 ** (stage + 2),j))(style)
                x = getattr(self, 'block_{0}_{1}'.format(2 ** (stage + 2),j))(x,affined_style,shape = (1,-1,2 ** (stage + 1),2 ** (stage + 1)))
                x = F.leaky_relu(x,LReLU_alpha)
                #x = x + torch.randn(x.shape, device = device) * getattr(self, 'noise_{0}_{1}'.format(2 ** (i + 2),j))
            x = getattr(self, 'to_rgb_{0}'.format(2 ** (stage + 2)))(x)
            if web:
                x_out = alpha * x + alternative_Upsample(x_out,(1,-1,2 ** (stage + 1),2 ** (stage + 1))) * (1 - alpha)
            else:
                x_out = alpha * x + torch.nn.functional.interpolate(x,scale_factor=2, mode='nearest') * (1 - alpha)
            
        return x_out

Using cuda device


In [575]:
# Define model
from torch.autograd import Variable
from torch import autograd
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        dimensions = [256,256,256,128,128,128,64,64,64,32,32,32]

        self.from_rgb_4 = nn.Conv2d(3,dimensions[2],kernel_size=1,stride=1,padding=0)
        self.block_4_0 = self.generate_block(4,dimensions[2],dimensions[1])
        self.block_4_1 = self.generate_block(4,dimensions[1],dimensions[0])

        self.from_rgb_8 = nn.Conv2d(3,dimensions[4],kernel_size=1,stride=1,padding=0)
        self.block_8_0 = self.generate_block(8,dimensions[4],dimensions[3])
        self.block_8_1 = self.generate_block(8,dimensions[3],dimensions[2])

        self.from_rgb_16 = nn.Conv2d(3,dimensions[6],kernel_size=1,stride=1,padding=0)
        self.block_16_0 = self.generate_block(16,dimensions[6],dimensions[5])
        self.block_16_1 = self.generate_block(16,dimensions[5],dimensions[4])

        self.from_rgb_32 = nn.Conv2d(3,dimensions[8],kernel_size=1,stride=1,padding=0)
        self.block_32_0 = self.generate_block(32,dimensions[8],dimensions[7])
        self.block_32_1 = self.generate_block(32,dimensions[7],dimensions[6])

        self.from_rgb_64 = nn.Conv2d(3,dimensions[10],kernel_size=1,stride=1,padding=0)
        self.block_64_0 = self.generate_block(64,dimensions[10],dimensions[9])
        self.block_64_1 = self.generate_block(64,dimensions[9],dimensions[8])

        self.final_conv = nn.Conv2d(dimensions[0],dimensions[0],kernel_size=4,stride=1,padding=0)
        self.linear = nn.Linear(dimensions[0],1)
        print(self.final_conv.weight)
        nn.init.normal_(self.final_conv.weight, 0.0, 1.0 / (4*4**dimensions[0]) ** 0.5)
        nn.init.constant_(self.final_conv.bias, 0)
        nn.init.normal_(self.linear.weight, 0.0, 1.0)
        nn.init.constant_(self.linear.bias, 0)
        for i in range(5):
            nn.init.normal_(getattr(self, 'from_rgb_{0}'.format(2 ** (i + 2))).weight, 0.0, 1.0 / getattr(self, 'from_rgb_{0}'.format(2 ** (i + 2))).weight.shape[1] ** 0.5)
            nn.init.constant_(getattr(self, 'from_rgb_{0}'.format(2 ** (i + 2))).bias, 0.0)

    def generate_block(self,image_size = 4,in_dimension = 512, out_dimension = 512):
        block = nn.Sequential()

        block.add_module('Conv', nn.Conv2d(in_dimension,out_dimension,kernel_size=3,stride=1,padding=1))
        block.add_module('relu', nn.LeakyReLU(LReLU_alpha))
        nn.init.normal_(block.Conv.weight, 0.0, 1.0)
        nn.init.constant_(block.Conv.bias, 0)

        return block
    #https://github.com/Zeleni9/pytorch-wgan/blob/master/models/wgan_gradient_penalty.py を改変
    def calculate_gradient_penalty(self, real_images, fake_images,batch_size,stage,alpha):
        eta = torch.FloatTensor(batch_size,1,1,1).uniform_(0,1).to(device)
        eta = eta.expand(batch_size, real_images.size(1), real_images.size(2), real_images.size(3))

        interpolated = eta * real_images + ((1 - eta) * fake_images)

        # define it to calculate gradient
        interpolated = Variable(interpolated, requires_grad=True)
        # calculate probability of interpolated examples
        prob_interpolated = self(interpolated,stage,alpha)
        # calculate gradients of probabilities with respect to examples
        gradients = autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                              grad_outputs=torch.ones(
                                  prob_interpolated.size()).to(device),
                              create_graph=True, retain_graph=True)[0]
        grad_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
        return grad_penalty
    
    def forward(self, image, stage = 1 ,alpha = 0, batches = 1):
        if alpha != 0:
            x = getattr(self, 'from_rgb_{0}'.format(2 ** (stage + 2)))(image)
            for j in range(2):
                x = getattr(self, 'block_{0}_{1}'.format(2 ** (stage + 2),j))(x)
            x = torch.nn.functional.interpolate(x,scale_factor=0.5, mode='nearest')
            x2 = getattr(self, 'from_rgb_{0}'.format(2 ** (stage + 1)))(torch.nn.functional.interpolate(image,scale_factor=0.5, mode='nearest'))
            x = x * alpha + x2 * (1 - alpha)
        else:
            x = getattr(self, 'from_rgb_{0}'.format(2 ** (stage + 1)))(image)

        for i in range(stage):
            for j in range(2):
                if not (i == stage - 1 and j == 1):
                    x = getattr(self, 'block_{0}_{1}'.format(2 ** (stage + 1 - i),j))(x)
            if i != stage - 1:
                x = torch.nn.functional.interpolate(x,scale_factor=0.5, mode='nearest')

        x = self.final_conv(x)
        
        x = nn.LeakyReLU(LReLU_alpha)(x)
        x = nn.Flatten()(x)
        return  self.linear(x)

In [576]:
class trainer():
    def __init__(self,learning_rate = 0.0002):
        self.alpha = 0
        self.stage = 0
        self.MAX_STAGE = 4
        self.BATCH_SIZE = 32
        self.n_critic = 1
        self.gp_lamda = 1
        self.g = Generator().to(device)
        self.d = Discriminator().to(device)
        self.G_optimizer = torch.optim.Adam(self.g.parameters(), lr=learning_rate)
        self.D_optimizer = torch.optim.Adam(self.d.parameters(), lr=learning_rate)
        self.NUM_EPOCH_NO_ALPHA = 50
        self.NUM_EPOCH_WITH_ALPHA = 50
        self.z_dimension = 256
    def train(self):
        MAX_STAGE = self.MAX_STAGE
        BATCH_SIZE = self.BATCH_SIZE
        n_critic = self.n_critic
        gp_lamda = self.gp_lamda
        g = self.g
        d = self.d
        G_optimizer = self.G_optimizer
        D_optimizer = self.D_optimizer
        NUM_EPOCH_NO_ALPHA = self.NUM_EPOCH_NO_ALPHA
        NUM_EPOCH_WITH_ALPHA = self.NUM_EPOCH_WITH_ALPHA
        z_dimension = self.z_dimension
        training_data = datasets.CIFAR10(
            root="data",
            train=True,
            download=True,
            transform=ToTensor(),
        )
        train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
        while True:
            for epoch in range(NUM_EPOCH_NO_ALPHA + NUM_EPOCH_WITH_ALPHA):
                number_of_batch = 0
                for X, _ in train_dataloader:
                    X = X * 2 - 1
                    
                    alpha = 0
                    if int(self.stage) % 2 == 1:
                        self.stage += 1 / (len(train_dataloader) * NUM_EPOCH_WITH_ALPHA) 
                    elif math.ceil(self.stage / 2) < self.MAX_STAGE:
                        self.stage += 1 / (len(train_dataloader) * NUM_EPOCH_NO_ALPHA)
                    stage = math.ceil(self.stage / 2)    
                    if int(self.stage) % 2 == 1:
                        alpha = self.stage - int(self.stage)
                    
                    for _ in range(n_critic):
                        z = torch.randn((BATCH_SIZE,z_dimension),device = device)
                        generated = g(z, stage = stage ,alpha = alpha, batches = BATCH_SIZE)
                        X = X.to(device)
                        X = torch.nn.functional.interpolate(X,size=generated.size(2), mode='Nearest')
                        y_real = d(X, stage = stage ,alpha = alpha, batches = BATCH_SIZE)
                        y_fake = d(generated, stage = stage ,alpha = alpha, batches = BATCH_SIZE)
                        d_loss = torch.mean(y_fake) - torch.mean(y_real) + d.calculate_gradient_penalty(X,generated,BATCH_SIZE,stage,alpha) * gp_lamda
                        Wasserstein_loss =  (torch.mean(y_fake) - torch.mean(y_real)).data

                        D_optimizer.zero_grad()
                        d_loss.backward()
                        D_optimizer.step()
                    g_loss = 'undefined'
                    if Wasserstein_loss <= 0:
                        z = torch.randn((BATCH_SIZE,z_dimension),device = device)
                        generated = g(z, stage = stage ,alpha = alpha, batches = BATCH_SIZE)
                        y_fake = d(generated, stage = stage ,alpha = alpha, batches = BATCH_SIZE)
                        g_loss = -torch.mean(y_fake)
                        G_optimizer.zero_grad()
                        g_loss.backward()
                        G_optimizer.step()

                    

                    if number_of_batch % 100 == 0:
                        print('epoch:{}, batch:{},stage:{},alpha{},g_loss:{}, d_loss:{}, Wasserstein Loss:{}'.format(epoch,number_of_batch,stage,alpha,g_loss,d_loss,Wasserstein_loss))
                        show_image(generated)
                        show_image(X)
                        torch.save(self, '/content/drive/MyDrive/StyleGAN2/model.pth')
                    number_of_batch += 1

In [577]:
try:
    train = torch.load('/content/drive/MyDrive/StyleGAN2/model.pth')
except:
    train = trainer()
    print('load failed')

dummy_input = torch.randn(1, train.z_dimension, device=device)
train.g.learning_const.requires_grad = False
for i in range(5):  
    for j in range(2):
        getattr(train.g, 'noise_{0}_{1}'.format(2 ** (i + 2),j)).requires_grad = False
        getattr(train.g, 'noise_{0}_{1}'.format(2 ** (i + 2),j)).requires_grad = False
stage = torch.tensor(2, dtype=torch.int)
alpha = torch.tensor(0, dtype=torch.int)
batches = torch.tensor(1, dtype=torch.int)
web = torch.tensor(True, dtype=torch.bool)
torch.onnx.export(train.g, (dummy_input,stage,alpha,batches,web), 'generator.onnx', opset_version= 9)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:129: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:133: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:143: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/usr/lo

In [578]:
from google.colab import files
files.download('generator.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [579]:
#train = torch.load('/content/drive/MyDrive/StyleGAN2/model.pth')
train = trainer()
z = torch.zeros(1, train.z_dimension, device=device)
train.g(z,web=True)

Parameter containing:
tensor([[[[ 4.1063e-04,  1.2369e-02,  6.2745e-03, -1.0765e-02],
          [-2.7792e-03, -1.3027e-02, -7.6648e-04, -1.2463e-02],
          [-7.0482e-03,  1.8424e-03, -1.2469e-02, -1.4241e-02],
          [ 1.2266e-02, -1.3108e-02,  1.0462e-03, -9.5371e-03]],

         [[-1.2973e-02,  9.4841e-03, -1.0359e-02,  7.0104e-03],
          [ 3.7099e-03, -8.1292e-03,  1.1262e-02, -7.1325e-03],
          [ 1.1717e-02,  1.1271e-02,  1.2432e-02,  1.4284e-02],
          [ 1.1957e-03, -5.3757e-04, -1.0045e-02, -5.1959e-03]],

         [[ 1.4351e-02, -2.3700e-03,  5.5031e-03, -7.4885e-03],
          [ 9.9936e-04, -4.5484e-03,  7.7253e-04, -1.2931e-02],
          [ 1.4001e-02, -7.2244e-03, -6.7728e-03, -1.3145e-02],
          [ 3.3331e-03,  3.2093e-03,  8.8484e-03,  3.2466e-03]],

         ...,

         [[-3.4181e-03,  1.4955e-02, -5.1061e-03,  1.0246e-02],
          [ 3.8197e-03,  3.2903e-03, -1.3589e-02, -4.7505e-03],
          [ 1.6433e-03, -1.2454e-02, -1.5099e-02,  9.1703e-03

tensor([[[[ 1.1413,  0.2075,  0.3919, -0.3964],
          [ 0.7401,  0.1210,  0.2165, -0.2177],
          [-0.2314,  0.6303,  0.9956, -0.5995],
          [ 0.8156,  0.7754,  0.3953, -0.4216]],

         [[ 0.5527,  0.8440,  0.4769,  0.4931],
          [ 0.2834,  0.5217,  0.5226, -0.0878],
          [-0.0370,  0.4777, -0.4944,  0.8379],
          [-0.0335,  0.7102,  0.7463,  0.4023]],

         [[-0.1886,  0.7285, -0.2532,  0.5262],
          [-0.9401,  0.3649,  0.4857,  0.1959],
          [-0.1238, -0.8521, -0.1096,  0.2135],
          [-0.0262,  0.9493, -0.4568,  0.3329]]]], device='cuda:0',
       grad_fn=<AddBackward0>)